In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf 
yf.pdr_override() 

In [24]:
class Portfolio():
    
    # GENERAL CAVEATS. This class assumes that:
    #     -The number of sold assest never exceds the number of 
    #      bought assets (i.e. I never sell what I don't own: no short-selling.)
    #     -I never spend more than the initial cash.
    #     -I can make multiple buys, but only one sell. Otherwise, I would have to modifiy 
    #      the computations for the Captital Gains Tax.
    
    
    '''
    Parameters
    ===========
    - cash: float. Initial cash position.
    
    Atributes
    ===========
    -cash: returns available cash after transactions
    -holdings: returns a pandas DataFrame with the information of every transacition. 
    
    Methods
    ===========
    - buy. Inputs: ticker (string), numShares (integer), price (real), tradeDate (string: '1900-01-30').
           Description: buys a number of a certain stock at a given price on a given date (optional).
           Returns: string. Description of the transaction.
    - sell. Inputs: ticker (string), numShares (integer), price (real), tradeDate (string: '1900-01-30').
            Description: sells a number of a certain stock at a given price on a given date (optional).
            Returns: string. Description of the transaction.
    - nav. Inputs: date (string: '1900-01-30').
           Description: Computes the Net Asset Value (NAV) of the porfolio.
           Returns: string with NAV on a given date, computed with the close price of the assets on that date. 
    - capitalGainsTax. Imputs: taxRate (float), method (string: 'Average','FIFO','LILO').
                       Description: Computes the Capital Gain Taxes due of the portfolio after a sale. 
    '''
    
    def __init__(self, cash):
        
        # attribute 'holdings' gives us a dataframe with updated information of 
        # each transaction in the order each transaction was made. 
        
        self.cash = cash
        self.holdings = pd.DataFrame(columns = ['Ticker','Number','Price', 'Date']) 
           
    def buy(self, ticker, numShares, price, date = None):
        
        # Method 'buy' returns a string with information of the buy transaction. 
        
        date = pd.to_datetime(date)
        invested = numShares*price
        available = self.cash - invested
        self.cash = available # We update the original cash MINUS the price we paid in the transaction. 
        self.holdings = self.holdings.append({'Ticker': ticker,'Number' : numShares,'Price': price, 'Date': date }, ignore_index=True)
        return 'Bought {} shares of {} at price {} each. You spent {}. Remaining Cash is {}.'.format(numShares, ticker, price, invested, available)
    
    def sell(self, ticker, numShares, price, date = None):
        
        # Method 'sell' returns a string with information of the sell transaction. 

        date = pd.to_datetime(date)
        gained = numShares*price
        available = self.cash + gained # We update the original cash PLUS the gain of the sale.
        self.cash = available
        self.holdings = self.holdings.append({'Ticker': ticker,'Number' : -numShares,'Price': price,'Date': date }, ignore_index=True)
        return 'Sold {} shares of {} at price {} each. You gained {}. Remaining Cash is {}.'.format(numShares, ticker,price, gained, available)
    
    def nav(self,date):
        
        # I compute 'nav' as: the sum over shares of the product of the close price of the share times the number 
        # of shares held, PLUS the available cash, all divided by the total number of shares:
        # ( Sum_{over all shares}(number*ClosePrice) + Cash )/(total number of shares)
        # https://www.investopedia.com/terms/n/nav.asp
        # Market values are dowloaded from Yahoo Finance. 
        
        names=list(set(self.holdings['Ticker']))
        start = pd.to_datetime(date) 
        end = pd.to_datetime(date) 
        market_prices= pdr.get_data_yahoo(names, start=start, end=end, progress=False)['Close']
        W = self.holdings['Number'].sum()
        S = 0
        for i in names: 
            S += self.holdings[self.holdings['Ticker'] == i]['Number'].sum()*market_prices[i][0]
        return 'NAV at date {} is {} per share.'.format(date, round((S+self.cash)/W,3))   
    
    def capitalGainsTax(self,taxRate, method):
        
        # This method computes the Capital Gain Taxes due of the portfolio. 
        
        names=list(set(self.holdings['Ticker']))
        cap_gain_sum = 0
        for tick in names:
            table_total = self.holdings[(self.holdings['Ticker']==tick)]
            table_buy = table_total[table_total['Number']>0]
            total_buy = table_buy['Number'].sum()
            total_sell = abs(table_total[table_total['Number']<0]['Number'].iloc[-1])
            sell_price = table_total[table_total['Number']<0]['Price'].iloc[-1]
            len_table = len(table_buy.index)
            
            # Method: Average
            # I compute the average buy prices for each asset by
            # summing number*buy_price and dividing by total number bought.
            if method == 'Average':
                num = 0
                for i in range(len_table):
                    num += table_buy['Number'].iloc[i]*table_buy['Price'].iloc[i]
                    buy_price = num/total_buy
                    if (sell_price - buy_price)>0: 
                        cap_gain = sell_price - buy_price
                    else:
                        cap_gain = 0 # If the capital gains for an asset are negative, I don't pay taxes for that asset.
                        
                cap_gain_sum += cap_gain
                tax = cap_gain_sum*taxRate
            
            # Method: FIFO
            # I compute the average buy prices for each asset by
            # substracting the number of sold assest from the list of bought ones starting from the
            # EARLIER ones, and averaging over the remaing. 
            if method == 'FIFO':
                book = pd.DataFrame({'Number':list(table_buy['Number']),'Reduced Number':list(table_buy['Number']),'Price':list(table_buy['Price'])})
                N=total_sell
                
                for j in book.index:
                    if book['Reduced Number'][j] < 0:
                        book['Reduced Number'][j] =0
                        break
                        
                for i in book.index:
                    book['Reduced Number'][i] = book['Number'][i]-N
                    N = abs(book['Number'][i]-N)
                    if book['Reduced Number'][i] >0:
                        break
                        
                    if  book['Reduced Number'][i] < 0:
                        book['Reduced Number'][i] =0 
                    
                num = 0
                total_buy = book['Reduced Number'].sum()
                for i in book.index:
                    num+= book['Reduced Number'][i]*book['Price'][i]
                    buy_price = num/total_buy
                    if (sell_price - buy_price)>0:
                        cap_gain = sell_price - buy_price
                    else:
                        cap_gain = 0 # If the capital gains for an asset are negative, I don't pay taxes for that asset.
                    
                cap_gain_sum += cap_gain
                tax = cap_gain_sum*taxRate
            
            # Method: LIFO
            # I compute the average buy prices for each asset by
            # substracting the number of sold assest from the list of bought ones starting from the
            # LATEST ones, and averaging over the remaing. 
            if method == 'LIFO':
                book = pd.DataFrame({'Number':list(table_buy['Number'])[::-1],'Reduced Number':list(table_buy['Number'])[::-1],'Price':list(table_buy['Price'])[::-1]})
                N=total_sell
                
                for j in book.index:
                    if book['Reduced Number'][j] < 0:
                        book['Reduced Number'][j] =0
                        break
                        
                for i in book.index:
                    book['Reduced Number'][i] = book['Number'][i]-N
                    N = abs(book['Number'][i]-N)
                    if book['Reduced Number'][i] >0:
                        break
                        
                    if  book['Reduced Number'][i] < 0:
                        book['Reduced Number'][i] =0 
                    
                num = 0
                total_buy = book['Reduced Number'].sum()
                for i in book.index:
                    num+= book['Reduced Number'][i]*book['Price'][i]
                    buy_price = num/total_buy
                    if (sell_price - buy_price)>0:
                        cap_gain = sell_price - buy_price
                    else:
                        cap_gain = 0 # If the capital gains for an asset are negative, I don't pay taxes for that asset.
                    
                cap_gain_sum += cap_gain
                tax = cap_gain_sum*taxRate

                
                
        return 'Capital gain taxes due are {} at {} tax rate. (Method:{})'.format(round(tax,3),taxRate,method)
      

# Test

In [42]:
port = Portfolio(10000)

In [43]:
port.cash

10000

In [44]:
port.buy('AAPL',7,203)

'Bought 7 shares of AAPL at price 203 each. You spent 1421. Remaining Cash is 8579.'

In [45]:
port.holdings

,Ticker,Number,Price,Date
0,AAPL,7,203,None


In [46]:
port.buy('AAPL',3,212)

'Bought 3 shares of AAPL at price 212 each. You spent 636. Remaining Cash is 7943.'

In [47]:
port.holdings

,Ticker,Number,Price,Date
0,AAPL,7,203,None
1,AAPL,3,212,None


In [48]:
port.cash

7943

In [49]:
port.buy('SHOP',15,388)

'Bought 15 shares of SHOP at price 388 each. You spent 5820. Remaining Cash is 2123.'

In [50]:
port.nav('2019-08-20')

'NAV at date 2019-08-20 is 394.046 per share.'

In [51]:
port.sell('AAPL',8,214)

'Sold 8 shares of AAPL at price 214 each. You gained 1712. Remaining Cash is 3835.'

In [52]:
port.holdings

,Ticker,Number,Price,Date
0,AAPL,7,203,None
1,AAPL,3,212,None
2,SHOP,15,388,None
3,AAPL,-8,214,None


In [53]:
port.sell('SHOP',5,385)

'Sold 5 shares of SHOP at price 385 each. You gained 1925. Remaining Cash is 5760.'

In [54]:
port.holdings

,Ticker,Number,Price,Date
0,AAPL,7,203,None
1,AAPL,3,212,None
2,SHOP,15,388,None
3,AAPL,-8,214,None
4,SHOP,-5,385,None


In [55]:
port.nav('2019-08-20')

'NAV at date 2019-08-20 is 827.535 per share.'

In [56]:
port.capitalGainsTax(0.1,method = 'Average')

'Capital gain taxes due are 0.83 at 0.1 tax rate. (Method:Average)'

In [57]:
port.capitalGainsTax(0.1,'FIFO')

'Capital gain taxes due are 0.2 at 0.1 tax rate. (Method:FIFO)'

In [58]:
port.capitalGainsTax(0.1,'LIFO')

'Capital gain taxes due are 1.1 at 0.1 tax rate. (Method:LIFO)'